In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse") \
  .getOrCreate()

:: loading settings :: url = jar:file:/opt/tljh/user/envs/spark/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter-jj/.ivy2/cache
The jars for the packages stored in: /home/jupyter-jj/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6ec2a0c7-07ee-467f-b99a-fbe3365b1157;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
:: resolution report :: resolve 91ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.ap

In [2]:
spark.sql('SHOW DATABASES').show()

+---------+
|namespace|
+---------+
|  default|
+---------+



# Create Table using SQL Query

In [3]:
spark.sql("""
    Create Table local.sales.Product(
        ProductId Int,
        Name String,
        Category String,
        Price Numeric,
        Stock Integer,
        IsActive Boolean
    )Using iceberg"""
)

DataFrame[]

In [4]:
spark.sql("""
  INSERT INTO local.sales.Product (ProductID, Name, Category, Price, Stock, IsActive) VALUES
(1, 'Laptop', 'Electronics', 999.99, 15, True),
(2, 'Smartphone', 'Electronics', 699.99, 30, True),
(3, 'Tablet', 'Electronics', 399.99, 20,True),
(4, 'Smartwatch', 'Wearable', 199.99, 25, True),
(5, 'Wireless Earbuds', 'Accessories', 129.99, 50,True),
(6, 'Bluetooth Speaker', 'Accessories', 89.99, 35, True),
(7, 'Gaming Console', 'Gaming', 499.99, 0, False),
(8, 'Mechanical Keyboard', 'Accessories', 79.99, 40,True),
(9, 'Gaming Mouse', 'Accessories', 59.99, 45, True),
(10, 'Monitor', 'Electronics', 249.99, 20, True),
(11, 'External Hard Drive', 'Storage', 119.99, 30, True),
(12, 'USB Flash Drive', 'Storage', 19.99, 0, False),
(13, 'Power Bank', 'Accessories', 39.99, 60, True),
(14, 'VR Headset', 'Gaming', 299.99, 8, True),
(15, 'Smart TV', 'Electronics', 799.99, 12, True),
(16, 'Wireless Router', 'Networking', 129.99, 22,True),
(17, 'Digital Camera', 'Photography', 549.99, 18, True),
(18, 'Tripod Stand', 'Photography', 49.99, 0, False),
(19, 'Fitness Tracker', 'Wearable', 89.99, 27, True),
(20, 'Electric Toothbrush', 'Health', 59.99, 40, True),
(21, 'Hair Dryer', 'Personal Care', 39.99, 28, True),
(22, 'Air Purifier', 'Home Appliances', 179.99, 0, False),
(23, 'Microwave Oven', 'Home Appliances', 249.99, 14, True),
(24, 'Washing Machine', 'Home Appliances', 499.99, 7, True),
(25, 'Refrigerator', 'Home Appliances', 999.99, 0, False);""")

DataFrame[]

In [5]:
spark.sql("""
    Select
    *
    From
    local.sales.Product""").show()

+---------+-------------------+-----------+-----+-----+--------+
|ProductId|               Name|   Category|Price|Stock|IsActive|
+---------+-------------------+-----------+-----+-----+--------+
|        1|             Laptop|Electronics| 1000|   15|    true|
|        2|         Smartphone|Electronics|  700|   30|    true|
|        3|             Tablet|Electronics|  400|   20|    true|
|        4|         Smartwatch|   Wearable|  200|   25|    true|
|        5|   Wireless Earbuds|Accessories|  130|   50|    true|
|        6|  Bluetooth Speaker|Accessories|   90|   35|    true|
|        7|     Gaming Console|     Gaming|  500|    0|   false|
|        8|Mechanical Keyboard|Accessories|   80|   40|    true|
|        9|       Gaming Mouse|Accessories|   60|   45|    true|
|       10|            Monitor|Electronics|  250|   20|    true|
|       11|External Hard Drive|    Storage|  120|   30|    true|
|       12|    USB Flash Drive|    Storage|   20|    0|   false|
|       13|         Power

# Create Table using Spark Syntax

In [8]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [9]:
schema = StructType([
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('job_tittle', StringType(), True)
])
data = [("Alice", 28, "HR"), ("Bob", 35, "Manager"), ("Charlie", 42, "Worker")]
df = spark.createDataFrame(data, schema=schema)

In [10]:
df.write.format("iceberg").mode("append").saveAsTable(f"local.sales.Employee")

In [11]:
iceberg_df = spark.read.format("iceberg").load(f"local.sales.Employee")

In [13]:
iceberg_df.show()

+-------+---+----------+
|   name|age|job_tittle|
+-------+---+----------+
|  Alice| 28|        HR|
|    Bob| 35|   Manager|
|Charlie| 42|    Worker|
+-------+---+----------+

